## Preprocesamiento

En este código se realiza la verificación de la validez de las coordenadas de latitud y longitud que se encuentran en los datos, así como la obtención de la información geográfica (condados y ciudad) a partir de las coordenadas.

In [1]:
## Paquetes/módulos
import pandas as pd

## Eliminar notación científica
pd.options.display.float_format = '{:.2f}'.format

## Directorio
dir = "C:/Users/ghama/Desktop/Minería de Datos/3 - Entregas/"

In [2]:
#==============================================#
#              Entrada de los datos            #
#==============================================#

## Datos de accidentes
datos = pd.read_csv(dir + "0 - Insumos/US_Accidents_Dec20.csv")
datos.head()


,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.00,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.87,-84.06,nan,nan,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.00,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.93,-82.83,nan,nan,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.00,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.06,-84.03,nan,nan,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.00,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.75,-84.21,nan,nan,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.00,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.63,-84.19,nan,nan,...,False,False,False,False,True,False,Day,Day,Day,Day


In [3]:
## Lista códigos TMC
tmclist = pd.read_table(dir + "0 - Insumos/tmc_list.txt", sep = ";")
tmclist.head()

,TMC,TMC_desc
0,1,traffic problem
1,2,queuing traffic (with average speeds Q). Dange...
2,11,overheight warning system triggered
3,12,"(Q) accident(s), traffic being directed around..."
4,16,"closed, rescue and recovery work in progress"


### Unión de la descripción para TMC con la descripción del accidente

In [4]:
## Unir variables de TMC y description
datos_new = datos[["ID","TMC","Description"]]

## Unir descripción de códigos TMC
datos_new = datos_new.join(tmclist.set_index("TMC"), how = "left", on = "TMC")

## Unir las dos columnas
datos_new["Description_TMC"] = datos_new["Description"] + datos_new["TMC_desc"]

## Eliminar columnas que no sirven
datos_new = datos_new[["ID","TMC","Description_TMC"]]
print(datos_new.shape)
datos_new.head()

(4232541, 3)


,ID,TMC,Description_TMC
0,A-1,201.00,Right lane blocked due to accident on I-70 Eas...
1,A-2,201.00,Accident on Brice Rd at Tussing Rd. Expect del...
2,A-3,201.00,Accident on OH-32 State Route 32 Westbound at ...
3,A-4,201.00,Accident on I-75 Southbound at Exits 52 52B US...
4,A-5,201.00,Accident on McEwen Rd at OH-725 Miamisburg Cen...


### Verificar la validez de las coordenadas de latitud y longitud

En primer lugar, Estados unidos se encuentra entre las latitudes 19.50139 a 64.85694 y longitudes -161.75583 a -68.01197. 

Así, como se puede apreciar más adelante, el mínimo y máximo en latitud que toman nuestras coordenadas es de 24.56 y 49.00, mientras que para la longitud tenemos valores que oscilan entre -124.62 y -67.11, por lo cual, todos estos valores caen en el rango mencionado anteriormente.

In [5]:
datos_new2 = datos[["ID","Start_Lat","Start_Lng"]]
datos_new2.head()

,ID,Start_Lat,Start_Lng
0,A-1,39.87,-84.06
1,A-2,39.93,-82.83
2,A-3,39.06,-84.03
3,A-4,39.75,-84.21
4,A-5,39.63,-84.19


In [6]:
datos_new2.describe()

,Start_Lat,Start_Lng
count,4232541.00,4232541.00
mean,36.40,-95.46
std,4.96,17.35
min,24.56,-124.62
25%,33.52,-117.36
50%,35.83,-90.02
75%,40.18,-80.85
max,49.00,-67.11


### Importar resultados

In [8]:
## Exportar resultados
datos_new.to_csv(dir + "0 - Insumos/datos_tmc.csv", sep = ";", header = True, index = False)